# **XRP**

In [11]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Binance API 연결
exchange = ccxt.binance({
    "rateLimit": 1200,
    "enableRateLimit": True,
})

# 심볼 및 시간 간격 설정
symbol = 'XRP/USDT'  # XRP를 USDT로 거래하는 데이터
timeframe = '5m'     # 5분 간격 데이터

# 가장 최신 데이터의 시작 날짜를 2024년 11월로 설정
start_timestamp = exchange.parse8601('2024-11-15T00:00:00Z')
data_limit = 10000  # 가져올 데이터 개수
ohlcv = []

# 데이터 수집
while len(ohlcv) < data_limit:
    try:
        new_data = exchange.fetch_ohlcv(symbol, timeframe, since=start_timestamp, limit=1000)
        if len(new_data) == 0:
            break
        ohlcv += new_data
        start_timestamp = new_data[-1][0] + 1  # 중복 방지
        time.sleep(1)  # API 제한 준수
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

# 데이터프레임으로 변환
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(ohlcv, columns=columns)

# 시간 변환 (유닉스 타임스탬프 -> 읽을 수 있는 시간)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 목표값 생성 (target_close: 5분 후의 close 값)
df['target_close'] = df['close'].shift(-1)

# 필요 없는 마지막 5개의 행 제거 (target_close가 NaN인 경우)
df = df[:-5]

# 이동 평균 (5분, 10분)
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_5_minus_ma_10'] = df['ma_5'] - df['ma_10']

# 표준편차 (5분)
df['std_5'] = df['close'].rolling(window=5).std()

# 볼린저 밴드
df['bollinger_high'] = df['ma_5'] + (df['std_5'] * 2)
df['bollinger_low'] = df['ma_5'] - (df['std_5'] * 2)

# RSI 계산
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], window=14)

# MACD 계산
short_ema = df['close'].ewm(span=12, adjust=False).mean()
long_ema = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = short_ema - long_ema
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# ATR (Average True Range)
df['true_range'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1)))
)
df['atr_5'] = df['true_range'].rolling(window=5).mean()

# 최근 상승/하락 비율
df['up_down_ratio'] = df['close'].diff().apply(lambda x: 1 if x > 0 else 0).rolling(window=5).mean()

# 종가와 최고가/최저가 간 거리
df['close_to_high_ratio'] = (df['close'] - df['low']) / (df['high'] - df['low'])
df['close_to_low_ratio'] = (df['high'] - df['close']) / (df['high'] - df['low'])

# 5분 변화율 (change_rate)
df['change_rate'] = df['close'].pct_change(periods=5)

# 필요 없는 첫 14개의 행 제거 (RSI 및 기타 계산에 필요한 초기 값이 NaN인 경우)
df = df.dropna()

# CSV 파일로 저장
output_file = 'XRP_data_2024_latest_4.csv'
df.to_csv(output_file, index=False)

print(f"CSV 파일로 저장 완료: {output_file}")

CSV 파일로 저장 완료: xrp_data_2024_latest_4.csv


# **BTC**

In [18]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Binance API 연결
exchange = ccxt.binance({
    "rateLimit": 1200,
    "enableRateLimit": True,
})

# 심볼 및 시간 간격 설정
symbol = 'BTC/USDT'
timeframe = '5m'     # 5분 간격 데이터

# 가장 최신 데이터의 시작 날짜를 2024년 11월로 설정
start_timestamp = exchange.parse8601('2024-10-20T00:00:00Z')
data_limit = 20000  # 가져올 데이터 개수
ohlcv = []

# 데이터 수집
while len(ohlcv) < data_limit:
    try:
        new_data = exchange.fetch_ohlcv(symbol, timeframe, since=start_timestamp, limit=1000)
        if len(new_data) == 0:
            break
        ohlcv += new_data
        start_timestamp = new_data[-1][0] + 1  # 중복 방지
        time.sleep(1)  # API 제한 준수
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

# 데이터프레임으로 변환
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(ohlcv, columns=columns)

# 시간 변환 (유닉스 타임스탬프 -> 읽을 수 있는 시간)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 목표값 생성 (target_close: 5분 후의 close 값)
df['target_close'] = df['close'].shift(-1)

# 필요 없는 마지막 5개의 행 제거 (target_close가 NaN인 경우)
df = df[:-5]

# 이동 평균 (5분, 10분)
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_5_minus_ma_10'] = df['ma_5'] - df['ma_10']

# 표준편차 (5분)
df['std_5'] = df['close'].rolling(window=5).std()

# 볼린저 밴드
df['bollinger_high'] = df['ma_5'] + (df['std_5'] * 2)
df['bollinger_low'] = df['ma_5'] - (df['std_5'] * 2)

# RSI 계산
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], window=14)

# MACD 계산
short_ema = df['close'].ewm(span=12, adjust=False).mean()
long_ema = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = short_ema - long_ema
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# ATR (Average True Range)
df['true_range'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1)))
)
df['atr_5'] = df['true_range'].rolling(window=5).mean()

# 최근 상승/하락 비율
df['up_down_ratio'] = df['close'].diff().apply(lambda x: 1 if x > 0 else 0).rolling(window=5).mean()

# 종가와 최고가/최저가 간 거리
df['close_to_high_ratio'] = (df['close'] - df['low']) / (df['high'] - df['low'])
df['close_to_low_ratio'] = (df['high'] - df['close']) / (df['high'] - df['low'])

# 5분 변화율 (change_rate)
df['change_rate'] = df['close'].pct_change(periods=5)

# 필요 없는 첫 14개의 행 제거 (RSI 및 기타 계산에 필요한 초기 값이 NaN인 경우)
df = df.dropna()

# CSV 파일로 저장
output_file = 'BTC_data_2024_latest_2.csv'
df.to_csv(output_file, index=False)

print(f"CSV 파일로 저장 완료: {output_file}")

CSV 파일로 저장 완료: btc_data_2024_latest2.csv


# **SOL**

In [22]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Binance API 연결
exchange = ccxt.binance({
    "rateLimit": 1200,
    "enableRateLimit": True,
})

# 심볼 및 시간 간격 설정
symbol = 'SOL/USDT'
timeframe = '5m'     # 5분 간격 데이터

# 가장 최신 데이터의 시작 날짜를 2024년 11월로 설정
start_timestamp = exchange.parse8601('2024-11-01T00:00:00Z')
data_limit = 20000  # 가져올 데이터 개수
ohlcv = []

# 데이터 수집
while len(ohlcv) < data_limit:
    try:
        new_data = exchange.fetch_ohlcv(symbol, timeframe, since=start_timestamp, limit=1000)
        if len(new_data) == 0:
            break
        ohlcv += new_data
        start_timestamp = new_data[-1][0] + 1  # 중복 방지
        time.sleep(1)  # API 제한 준수
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

# 데이터프레임으로 변환
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(ohlcv, columns=columns)

# 시간 변환 (유닉스 타임스탬프 -> 읽을 수 있는 시간)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 목표값 생성 (target_close: 5분 후의 close 값)
df['target_close'] = df['close'].shift(-1)

# 필요 없는 마지막 5개의 행 제거 (target_close가 NaN인 경우)
df = df[:-5]

# 이동 평균 (5분, 10분)
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_5_minus_ma_10'] = df['ma_5'] - df['ma_10']

# 표준편차 (5분)
df['std_5'] = df['close'].rolling(window=5).std()

# 볼린저 밴드
df['bollinger_high'] = df['ma_5'] + (df['std_5'] * 2)
df['bollinger_low'] = df['ma_5'] - (df['std_5'] * 2)

# RSI 계산
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], window=14)

# MACD 계산
short_ema = df['close'].ewm(span=12, adjust=False).mean()
long_ema = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = short_ema - long_ema
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# ATR (Average True Range)
df['true_range'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1)))
)
df['atr_5'] = df['true_range'].rolling(window=5).mean()

# 최근 상승/하락 비율
df['up_down_ratio'] = df['close'].diff().apply(lambda x: 1 if x > 0 else 0).rolling(window=5).mean()

# 종가와 최고가/최저가 간 거리
df['close_to_high_ratio'] = (df['close'] - df['low']) / (df['high'] - df['low'])
df['close_to_low_ratio'] = (df['high'] - df['close']) / (df['high'] - df['low'])

# 5분 변화율 (change_rate)
df['change_rate'] = df['close'].pct_change(periods=5)

# 필요 없는 첫 14개의 행 제거 (RSI 및 기타 계산에 필요한 초기 값이 NaN인 경우)
df = df.dropna()

# CSV 파일로 저장
output_file = 'SOL_data_2024_latest_1.csv'
df.to_csv(output_file, index=False)

print(f"CSV 파일로 저장 완료: {output_file}")

CSV 파일로 저장 완료: SOL_data_2024_latest_1.csv


# **ETH**

In [25]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Binance API 연결
exchange = ccxt.binance({
    "rateLimit": 1200,
    "enableRateLimit": True,
})

# 심볼 및 시간 간격 설정
symbol = 'ETH/USDT'
timeframe = '5m'     # 5분 간격 데이터

# 가장 최신 데이터의 시작 날짜를 2024년 11월로 설정
start_timestamp = exchange.parse8601('2024-11-01T00:00:00Z')
data_limit = 20000  # 가져올 데이터 개수
ohlcv = []

# 데이터 수집
while len(ohlcv) < data_limit:
    try:
        new_data = exchange.fetch_ohlcv(symbol, timeframe, since=start_timestamp, limit=1000)
        if len(new_data) == 0:
            break
        ohlcv += new_data
        start_timestamp = new_data[-1][0] + 1  # 중복 방지
        time.sleep(1)  # API 제한 준수
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

# 데이터프레임으로 변환
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(ohlcv, columns=columns)

# 시간 변환 (유닉스 타임스탬프 -> 읽을 수 있는 시간)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 목표값 생성 (target_close: 5분 후의 close 값)
df['target_close'] = df['close'].shift(-1)

# 필요 없는 마지막 5개의 행 제거 (target_close가 NaN인 경우)
df = df[:-5]

# 이동 평균 (5분, 10분)
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_5_minus_ma_10'] = df['ma_5'] - df['ma_10']

# 표준편차 (5분)
df['std_5'] = df['close'].rolling(window=5).std()

# 볼린저 밴드
df['bollinger_high'] = df['ma_5'] + (df['std_5'] * 2)
df['bollinger_low'] = df['ma_5'] - (df['std_5'] * 2)

# RSI 계산
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], window=14)

# MACD 계산
short_ema = df['close'].ewm(span=12, adjust=False).mean()
long_ema = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = short_ema - long_ema
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# ATR (Average True Range)
df['true_range'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1)))
)
df['atr_5'] = df['true_range'].rolling(window=5).mean()

# 최근 상승/하락 비율
df['up_down_ratio'] = df['close'].diff().apply(lambda x: 1 if x > 0 else 0).rolling(window=5).mean()

# 종가와 최고가/최저가 간 거리
df['close_to_high_ratio'] = (df['close'] - df['low']) / (df['high'] - df['low'])
df['close_to_low_ratio'] = (df['high'] - df['close']) / (df['high'] - df['low'])

# 5분 변화율 (change_rate)
df['change_rate'] = df['close'].pct_change(periods=5)

# 필요 없는 첫 14개의 행 제거 (RSI 및 기타 계산에 필요한 초기 값이 NaN인 경우)
df = df.dropna()

# CSV 파일로 저장
output_file = 'ETH_data_2024_latest_1.csv'
df.to_csv(output_file, index=False)

print(f"CSV 파일로 저장 완료: {output_file}")

CSV 파일로 저장 완료: ETH_data_2024_latest_1.csv


# **ADA**

In [29]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Binance API 연결
exchange = ccxt.binance({
    "rateLimit": 1200,
    "enableRateLimit": True,
})

# 심볼 및 시간 간격 설정
symbol = 'ADA/USDT'
timeframe = '5m'     # 5분 간격 데이터

# 가장 최신 데이터의 시작 날짜를 2024년 11월로 설정
start_timestamp = exchange.parse8601('2024-11-01T00:00:00Z')
data_limit = 20000  # 가져올 데이터 개수
ohlcv = []

# 데이터 수집
while len(ohlcv) < data_limit:
    try:
        new_data = exchange.fetch_ohlcv(symbol, timeframe, since=start_timestamp, limit=1000)
        if len(new_data) == 0:
            break
        ohlcv += new_data
        start_timestamp = new_data[-1][0] + 1  # 중복 방지
        time.sleep(1)  # API 제한 준수
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

# 데이터프레임으로 변환
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(ohlcv, columns=columns)

# 시간 변환 (유닉스 타임스탬프 -> 읽을 수 있는 시간)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 목표값 생성 (target_close: 5분 후의 close 값)
df['target_close'] = df['close'].shift(-1)

# 필요 없는 마지막 5개의 행 제거 (target_close가 NaN인 경우)
df = df[:-5]

# 이동 평균 (5분, 10분)
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_5_minus_ma_10'] = df['ma_5'] - df['ma_10']

# 표준편차 (5분)
df['std_5'] = df['close'].rolling(window=5).std()

# 볼린저 밴드
df['bollinger_high'] = df['ma_5'] + (df['std_5'] * 2)
df['bollinger_low'] = df['ma_5'] - (df['std_5'] * 2)

# RSI 계산
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], window=14)

# MACD 계산
short_ema = df['close'].ewm(span=12, adjust=False).mean()
long_ema = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = short_ema - long_ema
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# ATR (Average True Range)
df['true_range'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1)))
)
df['atr_5'] = df['true_range'].rolling(window=5).mean()

# 최근 상승/하락 비율
df['up_down_ratio'] = df['close'].diff().apply(lambda x: 1 if x > 0 else 0).rolling(window=5).mean()

# 종가와 최고가/최저가 간 거리
df['close_to_high_ratio'] = (df['close'] - df['low']) / (df['high'] - df['low'])
df['close_to_low_ratio'] = (df['high'] - df['close']) / (df['high'] - df['low'])

# 5분 변화율 (change_rate)
df['change_rate'] = df['close'].pct_change(periods=5)

# 필요 없는 첫 14개의 행 제거 (RSI 및 기타 계산에 필요한 초기 값이 NaN인 경우)
df = df.dropna()

# CSV 파일로 저장
output_file = 'ADA_data_2024_latest_1.csv'
df.to_csv(output_file, index=False)

print(f"CSV 파일로 저장 완료: {output_file}")

CSV 파일로 저장 완료: ADA_data_2024_latest_1.csv


# **BCH**

In [9]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Binance API 연결
exchange = ccxt.binance({
    "rateLimit": 1200,
    "enableRateLimit": True,
})

# 심볼 및 시간 간격 설정
symbol = 'BCH/USDT'
timeframe = '5m'     # 5분 간격 데이터

# 가장 최신 데이터의 시작 날짜를 2024년 11월로 설정
start_timestamp = exchange.parse8601('2024-11-01T00:00:00Z')
data_limit = 20000  # 가져올 데이터 개수
ohlcv = []

# 데이터 수집
while len(ohlcv) < data_limit:
    try:
        new_data = exchange.fetch_ohlcv(symbol, timeframe, since=start_timestamp, limit=1000)
        if len(new_data) == 0:
            break
        ohlcv += new_data
        start_timestamp = new_data[-1][0] + 1  # 중복 방지
        time.sleep(1)  # API 제한 준수
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

# 데이터프레임으로 변환
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(ohlcv, columns=columns)

# 시간 변환 (유닉스 타임스탬프 -> 읽을 수 있는 시간)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 목표값 생성 (target_close: 5분 후의 close 값)
df['target_close'] = df['close'].shift(-1)

# 필요 없는 마지막 5개의 행 제거 (target_close가 NaN인 경우)
df = df[:-5]

# 이동 평균 (5분, 10분)
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_5_minus_ma_10'] = df['ma_5'] - df['ma_10']

# 표준편차 (5분)
df['std_5'] = df['close'].rolling(window=5).std()

# 볼린저 밴드
df['bollinger_high'] = df['ma_5'] + (df['std_5'] * 2)
df['bollinger_low'] = df['ma_5'] - (df['std_5'] * 2)

# RSI 계산
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], window=14)

# MACD 계산
short_ema = df['close'].ewm(span=12, adjust=False).mean()
long_ema = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = short_ema - long_ema
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# ATR (Average True Range)
df['true_range'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1)))
)
df['atr_5'] = df['true_range'].rolling(window=5).mean()

# 최근 상승/하락 비율
df['up_down_ratio'] = df['close'].diff().apply(lambda x: 1 if x > 0 else 0).rolling(window=5).mean()

# 종가와 최고가/최저가 간 거리
df['close_to_high_ratio'] = (df['close'] - df['low']) / (df['high'] - df['low'])
df['close_to_low_ratio'] = (df['high'] - df['close']) / (df['high'] - df['low'])

# 5분 변화율 (change_rate)
df['change_rate'] = df['close'].pct_change(periods=5)

# 필요 없는 첫 14개의 행 제거 (RSI 및 기타 계산에 필요한 초기 값이 NaN인 경우)
df = df.dropna()

# CSV 파일로 저장
output_file = 'BCH_data_2024_latest_1.csv'
df.to_csv(output_file, index=False)

print(f"CSV 파일로 저장 완료: {output_file}")

CSV 파일로 저장 완료: BCH_data_2024_latest_1.csv


# **ONDO**

In [12]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Binance API 연결
exchange = ccxt.binance({
    "rateLimit": 1200,
    "enableRateLimit": True,
})

# 심볼 및 시간 간격 설정
symbol = 'ONDO/USDT'
timeframe = '5m'     # 5분 간격 데이터

# 가장 최신 데이터의 시작 날짜를 2024년 11월로 설정
start_timestamp = exchange.parse8601('2024-11-01T00:00:00Z')
data_limit = 20000  # 가져올 데이터 개수
ohlcv = []

# 데이터 수집
while len(ohlcv) < data_limit:
    try:
        new_data = exchange.fetch_ohlcv(symbol, timeframe, since=start_timestamp, limit=1000)
        if len(new_data) == 0:
            break
        ohlcv += new_data
        start_timestamp = new_data[-1][0] + 1  # 중복 방지
        time.sleep(1)  # API 제한 준수
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

# 데이터프레임으로 변환
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(ohlcv, columns=columns)

# 시간 변환 (유닉스 타임스탬프 -> 읽을 수 있는 시간)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 목표값 생성 (target_close: 5분 후의 close 값)
df['target_close'] = df['close'].shift(-1)

# 필요 없는 마지막 5개의 행 제거 (target_close가 NaN인 경우)
df = df[:-5]

# 이동 평균 (5분, 10분)
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_5_minus_ma_10'] = df['ma_5'] - df['ma_10']

# 표준편차 (5분)
df['std_5'] = df['close'].rolling(window=5).std()

# 볼린저 밴드
df['bollinger_high'] = df['ma_5'] + (df['std_5'] * 2)
df['bollinger_low'] = df['ma_5'] - (df['std_5'] * 2)

# RSI 계산
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], window=14)

# MACD 계산
short_ema = df['close'].ewm(span=12, adjust=False).mean()
long_ema = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = short_ema - long_ema
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# ATR (Average True Range)
df['true_range'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1)))
)
df['atr_5'] = df['true_range'].rolling(window=5).mean()

# 최근 상승/하락 비율
df['up_down_ratio'] = df['close'].diff().apply(lambda x: 1 if x > 0 else 0).rolling(window=5).mean()

# 종가와 최고가/최저가 간 거리
df['close_to_high_ratio'] = (df['close'] - df['low']) / (df['high'] - df['low'])
df['close_to_low_ratio'] = (df['high'] - df['close']) / (df['high'] - df['low'])

# 5분 변화율 (change_rate)
df['change_rate'] = df['close'].pct_change(periods=5)

# 필요 없는 첫 14개의 행 제거 (RSI 및 기타 계산에 필요한 초기 값이 NaN인 경우)
df = df.dropna()

# CSV 파일로 저장
output_file = 'ONDO_data_2024_latest_1.csv'
df.to_csv(output_file, index=False)

print(f"CSV 파일로 저장 완료: {output_file}")

CSV 파일로 저장 완료: ONDO_data_2024_latest_1.csv


# **BRETT**

In [15]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Binance API 연결
exchange = ccxt.binance({
    "rateLimit": 1200,
    "enableRateLimit": True,
})

# 심볼 및 시간 간격 설정
symbol = 'BRETT/USDT'
timeframe = '5m'     # 5분 간격 데이터

# 가장 최신 데이터의 시작 날짜를 2024년 11월로 설정
start_timestamp = exchange.parse8601('2024-11-01T00:00:00Z')
data_limit = 20000  # 가져올 데이터 개수
ohlcv = []

# 데이터 수집
while len(ohlcv) < data_limit:
    try:
        new_data = exchange.fetch_ohlcv(symbol, timeframe, since=start_timestamp, limit=1000)
        if len(new_data) == 0:
            break
        ohlcv += new_data
        start_timestamp = new_data[-1][0] + 1  # 중복 방지
        time.sleep(1)  # API 제한 준수
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

# 데이터프레임으로 변환
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(ohlcv, columns=columns)

# 시간 변환 (유닉스 타임스탬프 -> 읽을 수 있는 시간)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 목표값 생성 (target_close: 5분 후의 close 값)
df['target_close'] = df['close'].shift(-1)

# 필요 없는 마지막 5개의 행 제거 (target_close가 NaN인 경우)
df = df[:-5]

# 이동 평균 (5분, 10분)
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_5_minus_ma_10'] = df['ma_5'] - df['ma_10']

# 표준편차 (5분)
df['std_5'] = df['close'].rolling(window=5).std()

# 볼린저 밴드
df['bollinger_high'] = df['ma_5'] + (df['std_5'] * 2)
df['bollinger_low'] = df['ma_5'] - (df['std_5'] * 2)

# RSI 계산
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], window=14)

# MACD 계산
short_ema = df['close'].ewm(span=12, adjust=False).mean()
long_ema = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = short_ema - long_ema
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# ATR (Average True Range)
df['true_range'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1)))
)
df['atr_5'] = df['true_range'].rolling(window=5).mean()

# 최근 상승/하락 비율
df['up_down_ratio'] = df['close'].diff().apply(lambda x: 1 if x > 0 else 0).rolling(window=5).mean()

# 종가와 최고가/최저가 간 거리
df['close_to_high_ratio'] = (df['close'] - df['low']) / (df['high'] - df['low'])
df['close_to_low_ratio'] = (df['high'] - df['close']) / (df['high'] - df['low'])

# 5분 변화율 (change_rate)
df['change_rate'] = df['close'].pct_change(periods=5)

# 필요 없는 첫 14개의 행 제거 (RSI 및 기타 계산에 필요한 초기 값이 NaN인 경우)
df = df.dropna()

# CSV 파일로 저장
output_file = 'BRETT_data_2024_latest_1.csv'
df.to_csv(output_file, index=False)

print(f"CSV 파일로 저장 완료: {output_file}")

CSV 파일로 저장 완료: BRETT_data_2024_latest_1.csv


# **데이터 뽑기**

In [16]:
import ccxt
import pandas as pd
import numpy as np
import json
from datetime import datetime

# Binance API 연결
exchange = ccxt.binance({
    "rateLimit": 1200,
    "enableRateLimit": True,
})

# 심볼 및 시간 간격 설정
symbol = 'ADA/USDT'
timeframe = '5m'     # 5분 간격 데이터

# 현재 시간 기준으로 최신 데이터 가져오기
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=None, limit=10)

# 데이터프레임으로 변환
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(ohlcv, columns=columns)

# timestamp를 datetime 형식으로 변환
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 이동 평균 (5분, 10분)
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_5_minus_ma_10'] = df['ma_5'] - df['ma_10']

# 표준편차 (5분)
df['std_5'] = df['close'].rolling(window=5).std()

# 볼린저 밴드
df['bollinger_high'] = df['ma_5'] + (df['std_5'] * 2)
df['bollinger_low'] = df['ma_5'] - (df['std_5'] * 2)

# RSI 계산
def calculate_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], window=14)

# MACD 계산
short_ema = df['close'].ewm(span=12, adjust=False).mean()
long_ema = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = short_ema - long_ema
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# ATR (Average True Range)
df['true_range'] = np.maximum(
    df['high'] - df['low'],
    np.maximum(abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1)))
)
df['atr_5'] = df['true_range'].rolling(window=5).mean()

# 최근 상승/하락 비율
df['up_down_ratio'] = df['close'].diff().apply(lambda x: 1 if x > 0 else 0).rolling(window=5).mean()

# 종가와 최고가/최저가 간 거리
df['close_to_high_ratio'] = (df['close'] - df['low']) / (df['high'] - df['low'])
df['close_to_low_ratio'] = (df['high'] - df['close']) / (df['high'] - df['low'])

# 5분 변화율 (change_rate)
df['change_rate'] = df['close'].pct_change(periods=5)

# 필요 없는 초기 NaN 제거
df = df.dropna()

# 최신 데이터 추출1
latest_data = df.iloc[-1].drop(['timestamp']).to_dict()

# JSON 출력
print(json.dumps(latest_data, indent=4))

{
    "open": 1.2511,
    "high": 1.2516,
    "low": 1.2511,
    "close": 1.2514,
    "volume": 19675.5,
    "ma_5": 1.25602,
    "ma_10": 1.25981,
    "ma_5_minus_ma_10": -0.003790000000000182,
    "std_5": 0.005838835500337549,
    "bollinger_high": 1.267697671000675,
    "bollinger_low": 1.2443423289993247,
    "rsi": 38.74045801526714,
    "macd": -0.0022070175610593434,
    "macd_signal": -0.0008441680646466558,
    "true_range": 0.000600000000000156,
    "atr_5": 0.007320000000000037,
    "up_down_ratio": 0.6,
    "close_to_high_ratio": 0.6,
    "close_to_low_ratio": 0.4,
    "change_rate": -0.008949077373881287
}
